In [3]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [4]:
def khan_crawling(start_date, end_date):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    start_date_ = "".join(start_date.split('.'))
    end_date_ = "".join(end_date.split('.'))
    eng_url_head = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q=kaist&sort=1&d1='
    kor_url_head = 'http://search.khan.co.kr/search.html?stb=khan&dm=5&q=카이스트&sort=1&d1='
    url_tail = str(start_date_)+'~'+str(end_date_)+'&pg='
    
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + url_tail + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.phArtc')) == 0:
                break
            elements = soup.select('.phArtc a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + url_tail + str(page)
        try:  
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            if len(soup.select('.phArtc')) == 0:
                break
            elements = soup.select('.phArtc a[href]') # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            url1.append(link)
        page += 1
    url = list(set(url1).union(set(url2)))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link, headers=headers)
        html = response.text
        soup = bs(html,'html.parser')
        try:
            for script in soup(["script", "style"]):
                script.decompose() 
            title = soup.select('title')[0].get_text()
            content = soup.select('.art_body')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/kahn.xlsx",index = False)
    df.to_pickle("pickle files/kahn.pkl")

In [5]:
khan_crawling('2015.01.01', '2020.10.23')

87it [00:33,  2.62it/s]

Pandas(Index=86, url='http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201708301701001&code=940100')


103it [00:39,  2.96it/s]

Pandas(Index=102, url='http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201604301123001&code=940100')


108it [00:41,  2.03it/s]

...

1161it [07:07,  3.61it/s]

Pandas(Index=1160, url='http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201605290956001&code=920100')


1211it [07:24,  3.56it/s]

Pandas(Index=1210, url='http://news.khan.co.kr/kh_news/khan_art_view.html?artid=201610240030002&code=940100')


1218it [07:27,  2.72it/s]
